In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import functools
from six.moves import xrange  # pylint: disable=redefined-builtin


from tensorflow.data import Dataset, Iterator

import glob
import os,time
import PIL
import matplotlib.pyplot as plt
from scipy.misc import imresize


import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
from tqdm import trange, tqdm



from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

ValueError: tf.enable_eager_execution must be called at program startup.

In [10]:
print(tf.__version__)

1.9.0-rc2


A Tensor is a multi-dimensional array. Similar to NumPy ndarray objects, Tensor objects have a data type and a shape. Additionally, Tensors can reside in accelerator (like GPU) memory. TensorFlow offers a rich library of operations (tf.add, tf.matmul, tf.linalg.inv etc.) that consume and produce Tensors. These operations automatically convert native Python types.

In [11]:
print(tf.executing_eagerly())  
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))  

True
hello, [[4.]]


In [12]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


A tensor has both dtype and sizes

In [13]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


Conversion between tensorflow and numpy dtypes
TensorFlow operations automatically convert NumPy ndarrays to Tensors.
NumPy operations automatically convert Tensors to NumPy ndarrays.
Tensors can be explicitly converted to NumPy ndarrays by invoking the .numpy() method on them. These conversions are typically cheap as the array and Tensor share the underlying memory representation if possible. However, sharing the underlying representation isn't always possible since the Tensor may be hosted in GPU memory while NumPy arrays are always backed by host memory, and the conversion will thus involve a copy from GPU to host memory.

In [14]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [15]:
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
True
Is the Tensor on GPU #0:  
True


In [16]:
x

<tf.Tensor: id=90, shape=(3, 3), dtype=float32, numpy=
array([[0.02160025, 0.8575293 , 0.59637415],
       [0.6386746 , 0.35419858, 0.07505405],
       [0.71760154, 0.5081749 , 0.49256754]], dtype=float32)>

For tensorflow to use the data (images, text, sound, etc) we create tf.data.dataset objects

In [26]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""
            Line 1 
            Line 2
            Line 3
              """)

ds_file = tf.data.TextLineDataset(filename)

For images we do something similar, here is an example where we load CelebA dataset and create an appropriate dataset for the same 

In [30]:
celeba_dir = "../gan_assignment/data_for_gan_assignment/celebA/"











print('Loading CelebA from "%s"' % celeba_dir)
glob_pattern = os.path.join(celeba_dir,  '*.jpg')

image_filenames = sorted(glob.glob(glob_pattern))
print("Number of image files loaded: {} " .format(len(image_filenames)))



im = PIL.Image.open(image_filenames[100])
width, height = im.size

print("Images loaded have width {} and heigth {}".format(width,height)) 

def parse_function(filename):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    # image = tf.image.resize_images(image, [resize_size, resize_size])
    return image


def train_preprocess(image):
    image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label










## Setting up the data processing pipeline
batch_size = 128
dataset = tf.data.Dataset.from_tensor_slices(image_filenames)
dataset = dataset.shuffle(len(image_filenames))
dataset = dataset.map(parse_function, num_parallel_calls=4)
#dataset = dataset.map(train_preprocess, num_parallel_calls=4)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(2)

Loading CelebA from "../gan_assignment/data_for_gan_assignment/celebA/"
Number of image files loaded: 121072 
Images loaded have width 64 and heigth 64




### Gradient tape and its examples

Record operations for automatic differentiation.

Operations are recorded if they are executed within this context manager and at least one of their inputs is being "watched".

Trainable variables (created by tf.Variable or tf.get_variable, trainable=True is default in both cases) are automatically watched. Tensors can be manually watched by invoking the watch method on this context manager.

For example, consider the function y = x * x. The gradient at x = 3.0 can be computed as:

In [31]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    with tf.GradientTape() as gg:
        gg.watch(x)
        y = x * x
    dy_dx = gg.gradient(y, x)     # Will compute to 6.0
d2y_dx2 = g.gradient(dy_dx, x)  # Will compute to 2.0